In [1]:
import pandas as pd
from goatools.obo_parser import GODag

In [22]:
#load the go children data file
go_child_df = pd.read_csv("/Users/zaidur/Documents/Sequence_Project/aeromonasBact/uniprot_go_childs.csv")
#load the go parent database
go_parent_data = pd.read_csv("/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go_parent_data.csv")

In [24]:
go_child_df_head = go_child_df.head(5)

In [26]:
go_child_df_head

,id,sequence,go_terms
0,A0A068FVC1,MNIIKTAIPDVHIFEPKVFFDERGFFFESFNHKLFEEAVGYSVNFV...,"['GO:0008830', 'GO:0019305']"
1,A0A068FZD0,MTTQSSKSRVFVAGHRGMVGSAICRQLAQRTDIELVVRSRSELDLT...,"['GO:0016853', 'GO:0042351', 'GO:0050577', 'GO..."
2,A0A068FZK6,METSGLVAFVGTALAIACLRPLSAKLQLVDLPNQRKQHVGAIPLIG...,"['GO:0000287', 'GO:0005886', 'GO:0009243', 'GO..."
3,A0A075P9Z7,MNLTELKQKPITDLLQLAEEMGIENMARSRKQDVIFSLLKKHAKSG...,"['GO:0003723', 'GO:0004386', 'GO:0005524', 'GO..."
4,A0A075PBX8,MQISVNEFLTPRHIDVQVVSPTRAKITLEPLERGFGHTLGNALRRI...,"['GO:0000428', 'GO:0003677', 'GO:0003899', 'GO..."


In [36]:
#convert the go_terms column into list

import ast

go_child_df_head['go_terms'] = go_child_df_head['go_terms'].apply(ast.literal_eval)

/var/folders/8f/dbt9gbp93cbc6ns69dz8rq6c0000gn/T/ipykernel_38008/3552083970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  go_child_df_head['go_terms'] = go_child_df_head['go_terms'].apply(ast.literal_eval)


In [117]:
import pandas as pd
from goatools.obo_parser import GODag

#function for getting the ancestors and their generation level as a dictionary
def get_ancestors(go_term, ancestors_dict):
    ancestors_dict[go_term] = go_dag[go_term].level
    for parent in go_dag[go_term].parents:
        get_ancestors(parent.id, ancestors_dict)


# GO Database location
obo_fname = '/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go-basic.obo'

# Load the GO graph
go_dag = GODag(obo_file=obo_fname)

go_child_df_head['go_parents'] = '' # initialize new column

for index in range(len(go_child_df_head)):
    go_list = []
    for go_term in go_child_df_head.loc[index, 'go_terms']:
        ancestors_dict = {}
        get_ancestors(go_term, ancestors_dict)
        
        filtered_ancestors_dict = {k: v for k, v in ancestors_dict.items() if k in go_parent_data['go_id'].tolist()}

        # Get the highest level
        highest_level = max(filtered_ancestors_dict.values())

        # Get all ancestors with the highest level
        highest_level_ancestors = [k for k, v in filtered_ancestors_dict.items() if v == highest_level]
        
        go_list.extend(highest_level_ancestors)
        
    
    go_child_df_head.at[index, 'go_parents'] = go_list
    
        
    
go_child_df_head        

/Users/zaidur/Documents/Sequence_Project/aeromonasBact/go-basic.obo: fmt(1.2) rel(2023-05-10) 46,490 Terms


,id,sequence,go_terms,go_parents
0,A0A068FVC1,MNIIKTAIPDVHIFEPKVFFDERGFFFESFNHKLFEEAVGYSVNFV...,"[GO:0008830, GO:0019305]","[GO:0016853, GO:0055086, GO:1901135]"
1,A0A068FZD0,MTTQSSKSRVFVAGHRGMVGSAICRQLAQRTDIELVVRSRSELDLT...,"[GO:0016853, GO:0042351, GO:0050577, GO:0070401]","[GO:0016853, GO:0055086, GO:1901135, GO:001649..."
2,A0A068FZK6,METSGLVAFVGTALAIACLRPLSAKLQLVDLPNQRKQHVGAIPLIG...,"[GO:0000287, GO:0005886, GO:0009243, GO:000927...","[GO:0003674, GO:0005886, GO:0005975, GO:000561..."
3,A0A075P9Z7,MNLTELKQKPITDLLQLAEEMGIENMARSRKQDVIFSLLKKHAKSG...,"[GO:0003723, GO:0004386, GO:0005524, GO:000582...","[GO:0003723, GO:0003824, GO:0140657, GO:000367..."
4,A0A075PBX8,MQISVNEFLTPRHIDVQVVSPTRAKITLEPLERGFGHTLGNALRRI...,"[GO:0000428, GO:0003677, GO:0003899, GO:000573...","[GO:0005575, GO:0003677, GO:0140098, GO:000557..."
